## Initial configuration

In [24]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import cv2
import gc
from tqdm import tqdm,trange

import torch
from torchsummary import summary
from PIL import Image
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.optim as optim
import matplotlib.pyplot as plt

import urllib.request
import zipfile
import json

In [25]:
#Usefull for better outputs visualization
PURPLE = "\033[95m"
CYAN = "\033[96m"
DARKCYAN = "\033[36m"
BLUE = "\033[94m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
END = "\033[0m"

### Project settings

In [26]:
data_dir_unsplitted = "dataset/unsplitted"
save_dir_splitted = "dataset/splitted"
data_dir = "dataset/splitted"
models_dir ="model/"

learning_rate_e = 0.00001
learning_rate_d = 0.00001
learning_rate_c = 0.00001
num_epochs = 50

ALREADY_SPLITTED = False

#to be implemented:

LOAD_DATASET = True #if true "ALREADY_SPLITTED" is ignored
DATASET_ALREADY_DOWNLOADED = True 

LOAD_MODEL = False 
MODEL_ALREADY_DOWNLOADED = True 

## Dataset preparation

### Dataset splitting and data augmentation

In [27]:
classes = ["CLL", "FL", "MCL"]
id_to_name = {0:"CLL",1:"FL",2:"MCL"}

if(not ALREADY_SPLITTED or not LOAD_DATASET):
    data = []
    labels = []
    for i, cl in enumerate(classes):
        path = os.path.join(data_dir_unsplitted, cl)
        images = os.listdir(path)
        for img in tqdm(images,desc=cl):
            img_path = os.path.join(path, img)
            img = cv2.imread(img_path).astype(np.uint8)

#             #DATA AUGMENTATION

#             #flipping the image
#             flipped_img = np.fliplr(img)
#             #gaussian noise
#             noise = np.random.normal(0, 5, img.shape).astype(np.uint8)
#             noisy_img = img + noise

#             #adding the data and the labels to the lists
#             data.append(flipped_img)
#             labels.append(i)
#             data.append(noisy_img)
#             labels.append(i)
            data.append(img)
            labels.append(i) 


    data = np.array(data)
    labels = np.array(labels)


    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)


    train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.125, random_state=42)

    print("Train samples:", len(train_data))
    print("Test samples:", len(test_data))
    print("Validation samples:", len(val_data))

MCL: 100%|██████████████████████████████████████████████████████████████████████████| 122/122 [00:00<00:00, 131.56it/s]


Train samples: 261
Test samples: 75
Validation samples: 38


### Saving the splitted dataset

In [ ]:
#SAVING THE DATASET
if(not ALREADY_SPLITTED or not LOAD_DATASET):
    #TRAIN
    os.makedirs(os.path.join(save_dir_splitted,"train","images"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"train","labels"),exist_ok=True)

     
    labels_train_augmented={}
    for i, image in enumerate(tqdm(train_data,desc = "Train")):                       #images
        file_name = str(i)+"_train"+".png"
        path = os.path.join(save_dir_splitted,"train","images",file_name)
        labels_train_augmented[file_name] = train_labels[i]
        cv2.imwrite(path,image.astype(np.float64))
        
        #flipping
        flipped_img = np.fliplr(image)
        file_name = str(i)+"_flippend_train"+".png"
        path = os.path.join(save_dir_splitted,"train","images",file_name)
        labels_train_augmented[file_name] = train_labels[i]
        cv2.imwrite(path,flipped_img.astype(np.float64))
        
        #gaussian noise
        noise = np.random.normal(0, 5, img.shape).astype(np.uint8)
        noisy_image = image.astype(np.uint8) + noise
        file_name = str(i)+"_noisy_train"+".png"
        path = os.path.join(save_dir_splitted,"train","images",file_name)
        labels_train_augmented[file_name] = train_labels[i]
        cv2.imwrite(path,noisy_image.astype(np.float64))
    
    with open(os.path.join(save_dir_splitted,"train","labels","train.json"), 'w') as f:
        json.dump(labels_train_augmented, f)#labels

    #TEST
    os.makedirs(os.path.join(save_dir_splitted,"test","images"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"test","labels"),exist_ok=True)

    
    
    test_labels_dict = {}
    for i, image in enumerate(tqdm(test_data,desc = "Test")):                      #images
        file_name = str(i)+"_test"+".png"
        path = os.path.join(save_dir_splitted,"test","images",file_name)
        cv2.imwrite(path,image.astype(np.float64))
        test_labels_dict[file_name] = test_labels[i] 

    with open(os.path.join(save_dir_splitted,"test","labels","test.json"), 'w') as f:
        json.dump(test_labels_dict, f)#labels
        
    #VALIDATION
    os.makedirs(os.path.join(save_dir_splitted,"val","images"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"val","labels"),exist_ok=True)

    
    
    val_labels_dict = {}
    for i, image in enumerate(tqdm(val_data,desc = "Validation")):               #images
        file_name = str(i)+"_val"+".png"
        path = os.path.join(save_dir_splitted,"val","images",file_name)
        cv2.imwrite(path,image.astype(np.float64))
        val_labels_dict[file_name] = val_labels[i] 
        
    with open(os.path.join(save_dir_splitted,"val","labels","val.json"), 'w') as f:
        json.dump(val_labels_dict, f)#labels

Train:  74%|██████████████████████████████████████████████████████▎                  | 194/261 [01:00<00:19,  3.48it/s]

### Dataloaders definition

Notice that the labels are saved into a numpy array, while the images are read directly from the folder

In [22]:
class HDAdataset(Dataset):
    def __init__(self, root_dir, labels, train = False, transform=None):
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(os.listdir(self.root_dir))

    def __getitem__(self, idx):
        img_name = os.listdir(self.root_dir)[idx]
        img_path = os.path.join(self.root_dir, img_name)

        image = Image.open(img_path)
        image = np.asarray(image).astype(np.uint8)
        
        if self.train:
            image = image + np.random.normal(0, 2, image.shape).astype(np.uint8)
        
        #labels smoothing
        label = 0.1*np.ones(3)
        label[img_name] = 0.9
        
        if self.transform:
            image = self.transform((image/255.).astype(np.float32))

        return image, label

In [23]:
if(LOAD_DATASET and not DATASET_ALREADY_DOWNLOADED):
    print("download the dataset")
    urllib.request.urlretrieve("", "dataset.zip")
    with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
        zip_ref.extractall("./dataset/")


#convert to tensor each image
transform = transforms.Compose([transforms.ToTensor()])    

#TRAIN DATASET AND DATALOADER
train_dir = os.path.join(data_dir,"train","images")#images

train_labels = json.load(os.path.join(data_dir,"train","labels","train.json"))#labels

train_dataset = HDAdataset(train_dir,train_labels,train = False, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True,drop_last=True)

#TEST DATASET AND DATALOADER
test_dir = os.path.join(data_dir,"test","images")#images

test_labels = json.load(os.path.join(data_dir,"test","labels","test.json"))#labels

test_dataset = HDAdataset(test_dir,test_labels,train = False,transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=False)

#VALIDATION DATASET AND DATALOADER
val_dir = os.path.join(data_dir,"val","images")#images

val_labels = json.load(os.path.join(data_dir,"val","labels","val.json"))#labels

val_dataset = HDAdataset(val_dir,val_labels,train = False,transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=5, shuffle=False)

AttributeError: 'str' object has no attribute 'read'

## Neural network

In [8]:
torch.cuda.empty_cache()
gc.collect()

33

### Models definition

In [9]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(6),
            nn.LeakyReLU(),
            nn.Dropout(p=0.4),
            nn.Conv2d(in_channels=6, out_channels=9, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            nn.BatchNorm2d(9),
            nn.LeakyReLU(),
            nn.Dropout(p=0.4),
            nn.Conv2d(in_channels=9, out_channels=12, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            nn.BatchNorm2d(12),
            nn.LeakyReLU(),
            nn.Flatten(),
            nn.LeakyReLU(),
            nn.Dropout(p=0.4),
            nn.Linear(215760,10),
#             nn.BatchNorm1d(500),
#             nn.LeakyReLU(),
#             nn.Dropout(p=0.1),
#             nn.Linear(500,10),
            nn.BatchNorm1d(10),
            nn.LeakyReLU(),
            nn.Dropout(p=0.4),
            nn.Linear(10,3)
        )
        
        self.calassification = nn.Sequential(nn.Softmax(dim=1))

    def forward(self, x):
        x = self.encoder(x)
        return x, self.calassification(x)
    
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.decoder = nn.Sequential(
            nn.Linear(3,10),
            nn.Dropout(p=0.4),
#             nn.BatchNorm1d(500),
#             nn.LeakyReLU(),
#             nn.Dropout(p=0.1),
            nn.Linear(10,215760),
            nn.BatchNorm1d(215760),
            nn.LeakyReLU(),
            nn.Dropout(p=0.4),
            nn.Unflatten(-1, (12,116,155)),
            nn.ConvTranspose2d(in_channels=12, out_channels=9, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            nn.BatchNorm2d(9),
            nn.LeakyReLU(),
            nn.Dropout(p=0.4),
            nn.ConvTranspose2d(in_channels=9, out_channels=6, kernel_size=(3,3), stride=(3,3), padding=(0,1)),
            nn.BatchNorm2d(6),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(in_channels=6, out_channels=3, kernel_size=(3,3), stride=(1,1), padding=(1,2),dilation=2,output_padding=(0,1)),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.decoder(x)
        return x
    
# class Classifier(nn.Module):
#     def __init__(self):
#         super(Classifier, self).__init__()

#         self.classifier = nn.Sequential(
#             nn.Flatten(),
#             nn.BatchNorm1d(17980),
#             nn.LeakyReLU(),
#             nn.Dropout(p=0.4),
#             nn.Linear(17980,500),
#             nn.BatchNorm1d(500),
#             nn.LeakyReLU(),
#             nn.Dropout(p=0.4),
#             nn.Linear(500,10),
#             nn.BatchNorm1d(10),
#             nn.LeakyReLU(),
#             nn.Dropout(p=0.4),
#             nn.Linear(10,3),
#             nn.BatchNorm1d(3),
#             nn.Softmax(dim=1)
#         )


#     def forward(self, x):
#         x = self.classifier(x)
#         return x

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(BOLD+"You are working on: "+END+str(device))

You are working on: cuda


### Loading of the models

In [11]:
if(LOAD_MODEL and not MODEL_ALREADY_DOWNLOADED):
    print("download the model")

In [12]:
if(not LOAD_MODEL):
    encoder = Encoder().to(device)
    decoder = Decoder().to(device)
    #classifier = Classifier().to(device)
    
else:
    print("load the models from the folder")

In [13]:
#summary(encoder, (3, 1040, 1388),batch_size=3)
# summary(decoder, (1,3),batch_size=3)

### Training 

In [14]:
def test(dataloader,name):
    
    losses_ae = []
    losses_c = []
    count_correct = 0
    count_input = 0

    encoder.eval()
    decoder.eval()
    #classifier.eval()
    for data in tqdm(dataloader,desc = name +" set"):

        img, labels = data
        img = img.to(device)
        labels = labels.to(device)


        compressed_img,pred_labels = encoder(img)
        rec_img = decoder(compressed_img)
        #pred_labels = classifier(compressed_img)

        classifier_loss = criterion_classifier(pred_labels, labels)
        ae_loss= criterion_ae(img,rec_img)


        losses_ae.append(ae_loss.detach().cpu().numpy())
        losses_c.append(classifier_loss.detach().cpu().numpy())

        count_input+=pred_labels.detach().cpu().numpy().shape[0]
        count_correct+=np.sum((labels.to("cpu").detach().numpy().argmax(axis=1)==pred_labels.to("cpu").detach().numpy().argmax(axis=1)))
        del img
        del labels
        torch.cuda.empty_cache()
        gc.collect();

    # Print loss at end of each epoch
    accuracy= (count_correct/count_input)*100.
    print(name,' Autoencoder Loss: %.4f, Classifier Loss: %.4f, Accuracy: %.4f in %d input samples\n'
      % (np.mean(losses_ae), np.mean(losses_c),accuracy,count_input))
    
    return np.mean(losses_ae),np.mean(losses_c),accuracy

In [15]:
criterion_classifier = nn.CrossEntropyLoss()
criterion_ae = nn.MSELoss()

def train(train_dataloader,val_dataloader):
    optimizer_e = optim.Adam(encoder.parameters(), lr=learning_rate_e,weight_decay=1e-2)
    optimizer_d = optim.Adam(decoder.parameters(), lr=learning_rate_d,weight_decay=1e-2)
    #optimizer_c = optim.Adam(classifier.parameters(), lr=learning_rate_c,weight_decay=1e-2)
    
    
    best_epoch = 0 #(based on the validation accuracy)
    best_accuracy = -np.inf
    
    #for plotting
    train_loss_ae_trend = []
    train_loss_c_trend = []
    train_accuracy_trend =[]
    val_loss_ae_trend = []
    val_loss_c_trend = []
    val_accuracy_trend = []

    
    for epoch in range(num_epochs):

        ######training#####
        losses_ae = []
        losses_c = []
        count_correct = 0
        count_input = 0

        encoder.train()
        decoder.train()
        #classifier.train()
        for data in tqdm(train_dataloader,desc = "Train Epoch [%d/%d]"%(epoch+1,num_epochs)):



            img, labels = data
            img = img.to(device)
            labels = labels.to(device)


            optimizer_e.zero_grad()
            optimizer_d.zero_grad()
            #optimizer_c.zero_grad()


            compressed_img,pred_labels= encoder(img)
            rec_img = decoder(compressed_img)
            #pred_labels = classifier(compressed_img)

            classifier_loss = criterion_classifier(pred_labels, labels)
            ae_loss = criterion_ae(img,rec_img)
            total_loss = classifier_loss+0.01*ae_loss

#             classifier_loss.backward(retain_graph=True)
#             ae_loss.backward(retain_graph=True)

            total_loss.backward()
            optimizer_e.step()
            optimizer_d.step()
            #optimizer_c.step()


            losses_ae.append(ae_loss.detach().cpu().numpy())
            losses_c.append(classifier_loss.detach().cpu().numpy())

            count_input+=pred_labels.detach().cpu().numpy().shape[0]
            count_correct+=np.sum((labels.to("cpu").detach().numpy().argmax(axis=1)==pred_labels.to("cpu").detach().numpy().argmax(axis=1)))
            del img
            del labels
            torch.cuda.empty_cache()
            gc.collect();
            
        accuracy = (count_correct/count_input)*100.
        
        print('TRAIN: Epoch [%d/%d], Autoencoder Loss: %.4f, Classifier Loss: %.4f, Accuracy: %.4f in %d input samples \n'
              % (epoch+1, num_epochs, np.mean(losses_ae), np.mean(losses_c),accuracy,count_input))
        
        train_loss_ae_trend.append(np.mean(losses_ae))
        train_loss_c_trend.append(np.mean(losses_c))
        train_accuracy_trend.append(accuracy)
        
        ######validation######
        ae,c,acc = test(val_dataloader,"VALIDATION")
        
        val_loss_ae_trend.append(ae)
        val_loss_c_trend.append(c)
        val_accuracy_trend.append(acc)
        
        if acc > best_accuracy:
            print(GREEN+"Saved Model: "+END+UNDERLINE+"best validation accuracy reached\n\n"+END)
            best_epoch = epoch+1
            torch.save(encoder.state_dict(), "best_e.pt")
            torch.save(decoder.state_dict(), "best_d.pt")
            #torch.save(classifier.state_dict(), "best_c.pt")
            best_accuracy = acc

    return np.array([[train_loss_ae_trend,train_loss_c_trend,train_accuracy_trend],
            [val_loss_ae_trend,val_loss_c_trend,val_accuracy_trend],best_epoch],dtype=np.object_)

In [16]:
if(not LOAD_MODEL):
    results = train(train_dataloader,val_dataloader) 
    np.save("results.npy",results)

Train Epoch [1/50]: 100%|████████████████████████████████████████████████████████████| 156/156 [01:23<00:00,  1.87it/s]


TRAIN: Epoch [1/50], Autoencoder Loss: 0.0453, Classifier Loss: 1.2190, Accuracy: 33.7179 in 780 input samples 



VALIDATION set: 100%|████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.34it/s]


VALIDATION  Autoencoder Loss: 0.0270, Classifier Loss: 1.2178, Accuracy: 36.8421 in 38 input samples

Saved Model: best validation accuracy reached




Train Epoch [2/50]: 100%|████████████████████████████████████████████████████████████| 156/156 [01:17<00:00,  2.02it/s]


...

TRAIN: Epoch [3/50], Autoencoder Loss: 0.0443, Classifier Loss: 1.1928, Accuracy: 42.1795 in 780 input samples 



VALIDATION set: 100%|████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.37it/s]


VALIDATION  Autoencoder Loss: 0.0267, Classifier Loss: 1.2188, Accuracy: 18.4211 in 38 input samples



Train Epoch [4/50]:  73%|███████████████████████████████████████████▊                | 114/156 [00:57<00:21,  1.99it/s]


KeyboardInterrupt: 

### Test

Notice that the value of the losses and the accuracy for the train set are computed during the training, so they are not very accurate.

In [ ]:
train_dataset_no_noise = HDAdataset(train_dir,train_labels,train = False, transform=transform)
train_dataloader_no_noise = DataLoader(train_dataset_no_noise, batch_size=3, shuffle=False,drop_last=False)

In [ ]:
test(train_dataloader,"TRAIN")
test(val_dataloader,"VALIDATION")
test(test_dataloader,"TEST");